In [5]:
from selenium import webdriver
from PIL import Image
import io
import base64
import time
import re
from slugify import slugify
from webdriverdownloader import GeckoDriverDownloader

gdd = GeckoDriverDownloader()
geckodriver = gdd.download_and_install("v0.26.0")[1]

# See https://github.com/ouseful-template-repos/binder-selenium-demoscraper
# Note that currently isn't working on Binder -- perhaps related to this https://github.com/mozilla/geckodriver/issues/1559

In [6]:
URL = 'https://web.archive.org/web/19961019064223id_/http://www.nla.gov.au/'

def get_full_page_screenshot(url, capture_width=1200, save_width=200):
    options = webdriver.FirefoxOptions()
    options.headless = True
    
    driver = webdriver.Firefox(executable_path=geckodriver, options=options)
    driver.get(url)
    time.sleep(3)
    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(capture_width,S('Height')) # May need manual adjustment
    ss = driver.find_element_by_tag_name('body').screenshot_as_base64
    driver.quit()
    date_str, site = re.search(r'\/web\/(\d+)id_\/https*:\/\/(.+\/)', url).groups()
    img = Image.open(io.BytesIO(base64.b64decode(ss)))
    ratio = save_width / capture_width
    (width, height) = (int(img.width * ratio), int(img.height * ratio))
    resized_img = img.resize((width, height))
    resized_img.save(f'{slugify(site)}-{date_str}.png')

In [7]:
get_full_page_screenshot(URL)